In [1]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
mh = '+0.0'
CtoO = '1.0'


filename_db="/Users/sagnickmukherjee/Documents/GitHub/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
ck_db='/Users/sagnickmukherjee/Documents/software/picaso-dev/reference/opacities/ck_db/sonora_2020_feh+000_co_100.data.196'

opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db)


In [3]:
#[500,700, 800, 900, 1000,1300], g = [100, 178, 316, 562, 100], and log Kzz = [3,4, 5, 6, 7, 8,9]. 

T_eff= 1300
grav = 178
nlevel = 91

## Brown Dwarf 
T_star = "NA"
semi_major = "NA" # AU
self_consistent_kzz= False # Only needed if you want to do on-the-fly calculation with MLT Kzz
##


# Lets use Kzz = 1d4 in cgs

In [37]:
cl_run = jdi.inputs(calculation="brown", climate = True)

cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))
cl_run.T_eff(Teff=T_eff)




#pressure=np.logspace(np.log10(1.77000392e-04),np.log10(80.0),nlevel) # set your pressure grid
temp_guess = np.zeros(shape=(nlevel))
kz = np.zeros(shape =(nlevel))

## load a bobcat model
pressure_dummy,temp_guess = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(1300)+"g100"+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
pressure=pressure_dummy
for i in range(len(temp_guess)):
    temp_guess[i] = temp_guess[i] # perturb the profile
    kz[i] = 1e8 #0
    
nofczns = 1
nstr = np.array([0,65,89,0,0,0]) # initial guess of convective zones
rfacv = 0.0
rfaci =1.0
cloudy = False # not cloudy
fsed = 0.0

In [38]:
cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mh, CtoO = CtoO,species = ['MgSiO3','Mg2SiO4','Fe'], fsed = fsed)


In [ ]:
t0 = time.time()
pressure , temp, dtdp, nstr_new, flux_plus_final,quench_levels, df, all_profiles, all_kzz = cl_run.run_climate_model(opacity_ck,save_all_profiles=True, save_all_kzz = True,diseq_chem = True, self_consistent_kzz =self_consistent_kzz, kz = kz,vulcan_run = False)
t1 = time.time()

Iteration number  0 , min , max temp  631.8391425620954 3412.443067339931 , flux balance  0.00014122905260383317
Iteration number  1 , min , max temp  631.8275583231284 3412.3956505661736 , flux balance  3.3699043820375453e-06
Converged Solution in iterations  1
Big iteration is  631.8275583231284 0
Iteration number  0 , min , max temp  631.6663189697498 3412.0871326076267 , flux balance  2.4358197057561254e-05
Converged Solution in iterations  0
Profile converged
Iteration number  0 , min , max temp  631.9268721501463 3411.8272742186523 , flux balance  1.9371914903553835e-05
Converged Solution in iterations  0
Big iteration is  631.9268721501463 0
Iteration number  0 , min , max temp  631.8615791044324 3411.7545677638254 , flux balance  5.389022819358058e-06
Converged Solution in iterations  0
Profile converged
final [ 0 65 89  0  0  0]
Iteration number  0 , min , max temp  631.7959236595906 3411.75258654082 , flux balance  1.9440503508659027e-07
Converged Solution in iterations  0
Bi

In [ ]:
path = "/Users/sagnickmukherjee/Documents/research/geers/51erib/"

filename= path+"tpfile/tp_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_kzz_1e8.dat"
np.savetxt(filename, np.transpose([pressure,temp]), header= str("nstr "+str(nstr_new)+" T_eff "+str(T_eff)+" grav "+ str(grav)+" kz "+str(kz[0])+" T_star "+str(T_star)+" semi-major[AU] "+str(semi_major)))        

csv = path+"profile/profile_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_kzz_1e8.dat"
df.to_csv(csv,sep='\t')

hist = path+"hist/hist_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_kzz_1e8.dat"
np.savetxt(hist, all_profiles)

hist = path+"hist/hist_kz_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_kzz_1e8.dat"
np.savetxt(hist, all_kzz)

In [ ]:
pressure_bobcat,temp_bobcat = np.loadtxt("/Users/sagnickmukherjee/Documents/GitHub/Disequilibrium-Picaso/t"+str(1000)+"g1000"+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)

plt.figure(figsize=(10,10))
plt.ylabel("Pressure [Bars]", fontsize=25)
plt.xlabel('Temperature [K]', fontsize=25)
plt.ylim(205,1e-4)
plt.xlim(100,2000)

plt.semilogy(temp,pressure,color="r",linewidth=3,label="Our Run")


plt.minorticks_on()
plt.tick_params(axis='both',which='major',length =30, width=2,direction='in',labelsize=23)
plt.tick_params(axis='both',which='minor',length =10, width=2,direction='in',labelsize=23)

plt.legend(fontsize=15)

plt.title(r"T$_{\rm eff}$= 1000 K, log(g)=5.0",fontsize=25)

